In [ ]:
import wod_predictor
from wod_predictor.data_loader import DataLoader
from wod_predictor.splitter import DataSplitter
from wod_predictor.preprocessor import DataPreprocessor 
from wod_predictor.modeling import RandomForestModel,NeuralNetV0

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_path = wod_predictor.__path__[0].replace("wod_predictor", "Data")
loader = DataLoader(root_path = data_path, objects= ['open_results','descriptions','benchmark_stats',  'athlete_info'])
data = loader.load()

In [ ]:
splitter = DataSplitter(sample = 20000, test_ratio=  0.2, test_filter='23.*')
train_data, test_data = splitter.split(data)

In [ ]:
# Example using MinMaxScaler
preprocessing_config = {
    'open_results':
    {
        'scale_up': True,
        'scale_args': {
            "method": "general",
            "scaler_name": "MinMaxScaler"
        },
        'allow_modified' : False
    },
    'benchmark_stats':{
        'remove_outliers': True,
        'missing_method':'zero',
        'drop_missing_threshold': 0.9,
        'scale_args': {
            "method": "general",
            "scaler_name": "MinMaxScaler"
        }
        # 'neighbors': 5,
    },
    'athlete_info':{}
}
preprocessor = DataPreprocessor(config=preprocessing_config)
preprocessor.fit(data=train_data)


In [ ]:
train_preprocessed = preprocessor.transform(data=train_data)

In [ ]:
test_preprocessed = preprocessor.transform(data=test_data)

In [ ]:
#Neural Network

input_features_train = train_preprocessed['X'].shape[1]


model_V0 = NeuralNetV0(input_features = input_features_train, 
                       output_features = 1, 
                       hidden_units = 64)

model_V0.fit(train_preprocessed['X'], 
             train_preprocessed['y'], 
             epochs=1000, 
             lr=0.001)

y_pred = model_V0.predict(test_preprocessed['X'])

model_V0.show_results(y_test=test_preprocessed['y'], 
                      meta_data=test_preprocessed['meta_data'])
